# TCC TimeSeries Classification

In [9]:
import pandas as pd
import numpy as np


## Loading the data

In [17]:
df = pd.read_excel("./datasets/24PI109.xlsx", sheet_name= "Medições")

# index dataframe
df.set_index("DATE", inplace=True)

df

,COTA,MCA,PROF_NA
DATE,,,
2019-01-01,881.231000,NaN,NaN
2019-01-02,881.241000,14.140000,18.030000
2019-01-03,881.241000,14.140000,18.030000
2019-01-04,881.241000,14.140000,18.030000
2019-01-05,881.240000,14.139000,18.031000
...,...,...,...
2022-05-31,883.336585,16.235585,15.934415
2022-06-01,883.322128,16.221128,15.948872
2022-06-02,883.316206,16.215206,15.954794


## Data Preprocessing

### Inserting missing rows

In [20]:
# inserting missing day rows in dataframe
df = df.asfreq('D', fill_value=np.nan)

df

,COTA,MCA,PROF_NA
DATE,,,
2019-01-01,881.231000,NaN,NaN
2019-01-02,881.241000,14.140000,18.030000
2019-01-03,881.241000,14.140000,18.030000
2019-01-04,881.241000,14.140000,18.030000
2019-01-05,881.240000,14.139000,18.031000
...,...,...,...
2022-05-31,883.336585,16.235585,15.934415
2022-06-01,883.322128,16.221128,15.948872
2022-06-02,883.316206,16.215206,15.954794


### Interpolating the missing values

In [22]:
df = df.interpolate()

df.notna().sum()


COTA       1251
MCA        1250
PROF_NA    1250
dtype: int64

## X and no y

In [ ]:
X = df[["DATE", "MCA"]]


## Transformer model

In [23]:
# https://keras.io/examples/timeseries/timeseries_transformer_classification/
# https://colab.research.google.com/github/charlesollion/dlexperiments/blob/master/7-Transformers-Timeseries/Transformers_for_timeseries.ipynb
from tensorflow import keras
from tensorflow.keras import layers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 426.6/511.7 MB 1.0 MB/s eta 0:01:22
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    tensorflow from https://files.pythonhosted.org/packages/0b/3f/57bb9bbd2dfecba3659aedffb54bb837e83ccba81f217cb744f26d453048/tensorflow-2.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl#sha256=add2f4e55fd7209a5cb01d815364adc9827184d74a83c2f1616c72be2e85c8b7:
        Expected sha256 add2f4e55fd7209a5cb01d815364adc9827184d74a83c2f1616c72be2e85c8b7
             Got        8ae9e90f940d7bfa4bc37e425a7388254359331ec414bb65500c7e9871144d02



ModuleNotFoundError: No module named 'tensorflow'

### 

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)